# Import Required Libraries

In [1]:
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import MinMaxScaler, RobustScaler

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

from scipy.stats import ks_2samp

from model import OptimizedKMeans
from model import GeneticProfiling
from model import GeneticClustering
from model import DenoisingAutoencoder
from model import Dense
from model import ConvDense

from correlation import select_genes
from util import to_data_frame
from itertools import compress
from datetime import datetime

import lightgbm as lgb
import pandas as pd
import numpy as np
import pickle
import os

# Loading Data

In [2]:
clinical = pd.read_csv('data/clinical_brfl.tsv', sep='\t', index_col='ID')

genefpkm = pd.read_csv('data/gene_fpkm.tsv', sep='\t', index_col='ID')

selected_index = clinical.join(genefpkm, how='inner').index

clinical = clinical.loc[selected_index,:]

clinical['response_best_response_first_line'] = clinical['response_best_response_first_line'].astype(int)
therapy_class = clinical['therapy_first_line_class']
del clinical['therapy_first_line_class']

genefpkm = genefpkm.loc[selected_index,:]

# Transforming Qualitative Variables into Dummy Ones

In [3]:
for column in clinical:
    
    values = clinical[column]
    
    if values.dtype == 'object':
        
        values = pd.get_dummies(values)
        
        n_values = values.shape[1]
        
        values.columns = [column + '_' + str(c).lower().replace(' ', '_') for c in values.columns]
    
        del clinical[column]
        
        if n_values == 2:
            values = values.iloc[:, [0]]
        
        clinical = clinical.join(values, how='inner')

clinical = clinical.fillna(0)

clinical.iloc[:8,:]

,response_best_response_first_line,percent_aneuploid,percent_plama_cells_bone_marrow,percent_plama_cells_peripherical_blood,creatinine,iss,absolute_neutrophil,platelet,wbc_x10_10_9_l,bun,...,t_6_14_ccnd3_detected,t_8_14_mafa_detected,t_8_14_myc_detected,therapy_first_line_bor,therapy_first_line_bor-cyc-dex,therapy_first_line_bor-dex,therapy_first_line_bor-len-dex,therapy_first_line_len,therapy_first_line_len-dex,first_line_transplant_no
ID,,,,,,,,,,,,,,,,,,,,,
MMRF1029,0,0.0,8.4,0.0,106.080,1,2.60,219.0,4.0,5.355,...,0,0,0,0,0,0,1,0,0,1
MMRF1030,1,15.4,9.6,0.0,55.692,1,2.50,215.0,4.7,0.000,...,0,0,1,0,0,0,1,0,0,0
MMRF1031,0,18.3,10.1,0.0,81.328,1,10.29,385.0,12.4,4.284,...,0,0,0,0,0,0,1,0,0,1
MMRF1032,0,20.7,11.1,0.0,70.720,2,1.30,166.0,2.5,5.355,...,0,0,0,0,1,0,0,0,0,1
MMRF1033,0,18.5,12.0,0.0,79.560,1,3.99,307.0,7.4,4.998,...,0,0,0,0,0,0,0,0,1,1
MMRF1037,0,20.7,17.0,0.0,70.720,1,3.20,361.0,5.4,3.570,...,0,0,0,0,0,0,0,0,1,1
MMRF1038,0,29.0,22.0,0.0,97.240,3,5.89,310.0,10.5,7.854,...,0,0,0,0,0,0,0,0,1,1
MMRF1048,0,0.0,9.6,0.6,60.112,1,2.10,215.0,3.6,4.998,...,0,0,0,0,0,0,1,0,0,0


In [4]:
genefpkm.iloc[:8,:8]

,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036
ID,,,,,,,,
MMRF1029,17.651800,0.0,38.2082,6.61449,2.55321,0.237359,0.158369,55.2849
MMRF1030,1.595960,0.0,35.0286,4.62864,1.31648,0.642315,0.177913,40.7695
MMRF1031,0.030286,0.0,42.3708,5.34199,5.05216,0.439662,1.604640,63.2392
MMRF1032,0.681066,0.0,30.2787,3.01076,1.62869,0.587525,1.091920,20.0476
MMRF1033,0.595213,0.0,29.5319,6.10217,1.29061,2.331920,0.364109,25.0889
MMRF1037,0.513027,0.0,29.3762,4.32193,1.62923,0.131227,0.401753,26.2385
MMRF1038,5.238040,0.0,29.7864,3.88252,1.21612,0.393584,0.017117,26.4026
MMRF1048,1.131100,0.0,35.8575,3.54049,2.60006,0.663752,0.528562,18.1431


# Removing Bias from Therapy

In [5]:
sele = []

for c in clinical.columns:
    if 'therapy_first_line' in c:
        sele.append(c)

to_delete = []

for a in list(clinical[sele].loc[:,clinical[sele].sum() < 10].columns):
    to_delete += list(clinical.loc[clinical[a] == 1,:].index)
    del clinical[a]

clinical = clinical.loc[~clinical.index.isin(to_delete),:]

genefpkm = genefpkm.loc[~genefpkm.index.isin(to_delete),:]

# Clustering Analysis

In [6]:
from model import OptimizedKMeans
from collections import Counter

k = OptimizedKMeans()

bla = clinical.iloc[:,1:].join(genefpkm, how='inner')

sss = MinMaxScaler()

bla = sss.fit_transform(bla)

k.fit(bla)

k.n_clusters

6

In [7]:

Counter(k.predict(bla))

Counter({3: 84, 5: 98, 0: 76, 1: 56, 2: 50, 4: 57})

# Removing Bias from Clusters

In [8]:
#dddd = pd.DataFrame({'cluster': k.predict(clinical.iloc[:,1:].join(genefpkm, how='inner'))}, index=clinical.iloc[:,1:].join(genefpkm, how='inner').index)

#dddd

#genefpkm = genefpkm.loc[dddd['cluster'] != 0,:]
#clinical = clinical.loc[dddd['cluster'] != 0,:]

In [9]:
#Counter(k.predict(bla))

# Training Process

In [13]:
from collections import Counter
from scipy.special import erfinv
from sklearn.preprocessing import StandardScaler

kfold = StratifiedKFold(10, random_state=13)

result = None

#
#
#
x, y = clinical.values[:, 1:], clinical.values[:, 0]

for i, (train_index, valid_index) in enumerate(kfold.split(x, y)):
    
    print('Fold #{}'.format(i + 1))
    
    #
    # Split train & valid
    #
    response_train = clinical.iloc[train_index, [0]].values
    response_valid = clinical.iloc[valid_index, [0]].values
    
    clinical_scaler = RobustScaler()
    
    clinical_train = clinical.iloc[train_index, 1:]
    clinical_train = (clinical_train - clinical_train.min()) / (clinical_train.max() - clinical_train.min())
    clinical_train = pd.DataFrame(clinical_scaler.fit_transform(clinical_train.values), columns=clinical_train.columns, index=clinical_train.index)
    
    clinical_valid = clinical.iloc[valid_index, 1:]
    clinical_valid = (clinical_valid - clinical_train.min()) / (clinical_train.max() - clinical_train.min())
    clinical_valid = pd.DataFrame(clinical_scaler.transform(clinical_valid.values), columns=clinical_valid.columns, index=clinical_valid.index)
    
    genefpkm_scaler = RobustScaler()
    
    genefpkm_train = genefpkm.iloc[train_index, :]
    genefpkm_train = (genefpkm_train - genefpkm_train.min()) / (genefpkm_train.max() - genefpkm_train.min())
    genefpkm_train = pd.DataFrame(genefpkm_scaler.fit_transform(genefpkm_train.values), columns=genefpkm_train.columns, index=genefpkm_train.index)
    
    genefpkm_valid = genefpkm.iloc[valid_index, :]
    genefpkm_valid = (genefpkm_valid - genefpkm_train.min()) / (genefpkm_train.max() - genefpkm_train.min())
    genefpkm_valid = pd.DataFrame(genefpkm_scaler.transform(genefpkm_valid.values), columns=genefpkm_valid.columns, index=genefpkm_valid.index)
    
    #
    # Select gene expressions
    #
    print('Selecting gene expressions')
    
    if os.path.isfile('output/selected_genes_fold_{}.pkl'.format(i)):
        
        with open('output/selected_genes_fold_{}.pkl'.format(i), 'rb') as file:
            selected_genes = sorted(list(pickle.load(file)))
            
        with open('output/selected_feats_fold_{}.pkl'.format(i), 'rb') as file:
            selected_feats = sorted(list(pickle.load(file)))
    
    else:
        
        selected_genes = sorted(list(select_genes(genefpkm_train, response_train[:,0], threshold=0.02)))
        
        selected_feats = sorted(list(select_genes(clinical_train, response_train[:,0], threshold=0.2)))
        
        with open('output/selected_genes_fold_{}.pkl'.format(i), 'wb') as file:
            pickle.dump(selected_genes, file)
            
        with open('output/selected_feats_fold_{}.pkl'.format(i), 'wb') as file:
            pickle.dump(selected_feats, file)

    print('Selecting {} gene expressions'.format(len(selected_genes)))
    
    genefpkm_train = genefpkm_train.loc[:,selected_genes]
    clinical_train = clinical_train.loc[:,selected_feats]
    
    genefpkm_valid = genefpkm_valid.loc[:,selected_genes]
    clinical_valid = clinical_valid.loc[:,selected_feats]
    
    #
    # Genetic Profiling
    #
    print('Computing genetic profling')
    
    if os.path.isfile('output/kmeans_genetic_profiling_fold_{}.pkl'.format(i)):
        
        with open('output/kmeans_genetic_profiling_fold_{}.pkl'.format(i), 'rb') as file:
            genetic_profiling = pickle.load(file)
        
    else:
        
        genetic_profiling = GeneticProfiling(random_state=10)

        genetic_profiling.fit(genefpkm_train)
        
        with open('output/kmeans_genetic_profiling_fold_{}.pkl'.format(i), 'wb') as file:
            pickle.dump(genetic_profiling, file)
        
    
    profiling_train = to_data_frame(genetic_profiling.transform(genefpkm_train), prefix='PV', index=genefpkm_train.index)
    #profiling_train = to_data_frame(genetic_profiling.predict(genefpkm_train).reshape((-1,1)), prefix='PV', index=genefpkm_train.index)
    clinical_train = pd.concat([clinical_train, profiling_train], axis=1)
    
    profiling_valid = to_data_frame(genetic_profiling.transform(genefpkm_valid), prefix='PV', index=genefpkm_valid.index)
    #profiling_valid = to_data_frame(genetic_profiling.predict(genefpkm_valid).reshape((-1,1)), prefix='PV', index=genefpkm_valid.index)
    clinical_valid = pd.concat([clinical_valid, profiling_valid], axis=1)
    
    #
    # Gene Clustering
    #
    print('Computing genetic clustering')
    
    if os.path.isfile('output/kmeans_genetic_clustering_fold_{}.pkl'.format(i)):
        
        with open('output/kmeans_genetic_clustering_fold_{}.pkl'.format(i), 'rb') as file:
            genetic_clustering = pickle.load(file)
        
    else:
        
        genetic_clustering = GeneticClustering(random_state=10, verbose=0, early_stopping_rounds=10)

        genetic_clustering.fit(genefpkm_train)
        
        with open('output/kmeans_genetic_clustering_fold_{}.pkl'.format(i), 'wb') as file:
            pickle.dump(genetic_clustering, file)
    
    gene_cluster_train = to_data_frame(genetic_clustering.transform(genefpkm_train), prefix='GC', index=genefpkm_train.index)
    clinical_train = pd.concat([clinical_train, gene_cluster_train], axis=1)
    
    gene_cluster_valid = to_data_frame(genetic_clustering.transform(genefpkm_valid), prefix='GC', index=genefpkm_valid.index)
    clinical_valid = pd.concat([clinical_valid, gene_cluster_valid], axis=1)
    
    #
    # Denoising Autoencoder
    #
    print('Denoising autoencoder')
    
    dae = DenoisingAutoencoder(model_name='data_augmentation_adam_fold_{}'.format(i), summaries_dir='output/deep_models/', verbose=1)
    
    if not os.path.exists('output/deep_models/data_augmentation_adam_fold_{0}/graph/data_augmentation_adam_fold_{0}.meta'.format(i)):
        
        dae.build(n_inputs=x_train.shape[1], 
                  encoder_units=(int(genefpkm_train.values.shape[1] * .9), int(genefpkm_train.values.shape[1] * .8), int(genefpkm_train.values.shape[1] * .7)), 
                  decoder_units=(int(genefpkm_train.values.shape[1] * .8), int(genefpkm_train.values.shape[1] * .9)), 
                  encoder_activation_function='relu', decoder_activation_function='identity', l2_scale=0.01)
        
        dae.fit(genefpkm_train.values, steps=2500, optimizer='adam', loss='mse', learning_rate=1e-2)#, keep_probability=0.5)
        
    dae.load('output/deep_models/data_augmentation_adam_fold_{0}/graph/data_augmentation_adam_fold_{0}'.format(i))
    
    error_scaler = RobustScaler()
    
    error_train = dae.predict(genefpkm_train.values)
    error_train = error_scaler.fit_transform(error_scaler)
    error_train = pd.DataFrame(error_train, index=genefpkm_train.index)
    error_train.columns = ['ERR' + ens.replace('ENS', '') for ens in genefpkm_train.columns]
    
    error_valid = dae.predict(genefpkm_valid.values)
    error_valid = error_scaler.transform(error_valid)
    error_valid = pd.DataFrame(error_valid, index=genefpkm_valid.index)
    error_valid.columns = ['ERR' + ens.replace('ENS', '') for ens in genefpkm_valid.columns]
    
    #error_train = pd.DataFrame(dae.encode(x_train), index=genefpkm_train.index)
    #error_train = pd.DataFrame(np.abs(x_train - dae.predict(x_train)), index=genefpkm_train.index)
    #error_train = pd.DataFrame(dae.get_error(x_train), index=genefpkm_train.index)
    #error_train.columns = ['ERR' + str(pre) for pre in range(error_train.shape[1])]
    
    #error_valid = pd.DataFrame(dae.encode(x_valid), index=genefpkm_valid.index)
    #error_valid = pd.DataFrame(np.abs(x_valid - dae.predict(x_valid)), index=genefpkm_valid.index)
    #error_valid = pd.DataFrame(dae.get_error(x_valid), index=genefpkm_valid.index)
    #error_valid.columns = ['ERR' + str(pre) for pre in range(error_valid.shape[1])]
    
    dae.close()
    
    del dae
    
    #
    # Join all features
    #
   
    #x_train = erfinv(final_scalar.fit_transform(clinical_train.join(error_train, how='inner').fillna(0).values))
    #x_valid = erfinv(final_scalar.transform(clinical_valid.join(error_valid, how='inner').fillna(0).values)
    
    x_train = clinical_train.join(genefpkm_train, how='inner').join(error_train, how='inner').values
    x_valid = clinical_valid.join(genefpkm_valid, how='inner').join(error_valid, how='inner').values
    
    #
    # Dense
    #
    print('Multiple Dense Models')
    
    md_prefix = 'multiple_dense_selected_fold'
    
    multiple_dense = Dense(model_name='{}_{}'.format(md_prefix, i), summaries_dir='output/deep_models/')
    
    if not os.path.exists('output/deep_models/{0}_{1}/{0}_{1}.meta'.format(md_prefix, i)):
        
        multiple_dense.build(n_input_features=x_train.shape[1], 
                             n_outputs=1, 
                             abstraction_activation_functions=('sigmoid', 'tanh', 'relu'),
                             n_hidden_layers=3, n_hidden_nodes=int(x_train.shape[1] * .45) + 1, 
                             keep_probability=0.5,
                             optimizer_algorithms=('adam', 'adam', 'adam'), 
                             cost_function='logloss', 
                             add_summaries=True,
                             batch_normalization=True, l2_regularizer=1e-8)
        
        multiple_dense.fit(x_train, response_train, x_valid, 
                           response_valid, learning_rate=1e-2, 
                           steps=1000, batch_size=1000, shuffle=True)
    
    multiple_dense.load('output/deep_models/{0}_{1}/{0}_{1}'.format(md_prefix, i))

    x_train_transformed = multiple_dense.transform(x_train)
    x_valid_transformed = multiple_dense.transform(x_valid)
    
    multiple_dense.close()
    
    del multiple_dense
    
    #
    # Conv Dense
    #
    '''
    for ii in range(x_train_transformed.shape[1]):
        
        for jj in range(x_train_transformed.shape[2]):
            
            for kk in range(x_train_transformed.shape[3]):
                
                s = StandardScaler()
                
                s.fit(x_train_transformed[:,ii,jj,kk,0].reshape((-1, 1)))
                
                x_train_transformed[:,ii,jj,kk,0] = s.transform(x_train_transformed[:,ii,jj,kk,0].reshape((-1, 1))).reshape((-1))
                
                x_valid_transformed[:,ii,jj,kk,0] = s.transform(x_valid_transformed[:,ii,jj,kk,0].reshape((-1, 1))).reshape((-1))
            
    x_train_transformed = np.nan_to_num(x_train_transformed)
    x_valid_transformed = np.nan_to_num(x_valid_transformed)
    
    cd_prefix = 'convdense_standard_drop=0.5_lr=1e-4_nodecay_nonorm_huberloss'
    
    conv_dense = ConvDense(model_name='{}_fold_{}'.format(cd_prefix, i), summaries_dir='output/deep_models/', verbose=1)
    
    if not os.path.exists('output/deep_models/{0}_fold_{1}/graph/{0}_fold_{1}.meta'.format(cd_prefix, i)):
    
        conv_dense.build(n_models=3, n_neurons_per_layer=int(x_train.shape[1] * .45) + 1, n_layers=3, 
                         n_outputs=1, optimizer_algorithm='adagrad', keep_probability=0.5, loss='huber')

        conv_dense.fit(x_train_transformed, response_train, x_valid_transformed, response_valid, 
                       batch_size=x_train.shape[0], steps=30, learning_rate=1e-4)    
    
    conv_dense.load('output/deep_models/{0}_fold_{1}/graph/{0}_fold_{1}'.format(cd_prefix, i))
    
    x_train = conv_dense.transform(x_train_transformed)
    
    x_valid = conv_dense.transform(x_valid_transformed)
    
    conv_dense.close()
    
    del conv_dense
    '''
    s = x_train_transformed.shape
    x_train = x_train_transformed.reshape(-1, s[1] * s[2] * s[3])
    
    s = x_valid_transformed.shape
    x_valid = x_valid_transformed.reshape(-1, s[1] * s[2] * s[3])
    
    #
    #
    #  
    
    params = {'boosting_type': 'gbdt', 
              'objective': 'binary',
              'num_class': 1,
              'metric': ['auc', 'binary_logloss'],
              'learning_rate': 0.01, 
              'num_leaves': 10, 
              'max_depth': 4,  
              'min_child_samples': 10, 
              'max_bin': 100,  
              'subsample': 0.7, 
              'subsample_freq': 2,  
              'colsample_bytree': 0.1,  
              'min_child_weight': 3, 
              'subsample_for_bin': 10,
              'min_split_gain': 1, 
              'reg_alpha': 0, 
              'reg_lambda': 0, 
              'nthread': 26, 
              'verbose': 1}

    lgb_train = lgb.Dataset(x_train, list(response_train.reshape((-1,))))
    lgb_valid = lgb.Dataset(x_valid, list(response_valid.reshape((-1,))))
    
    gbm = lgb.train(params, lgb_train, num_boost_round=1000)    

    y_ = gbm.predict(x_valid, num_iteration=gbm.best_iteration)

    #
    #
    #
    auc = roc_auc_score(response_valid, y_)
    
    print(auc)
    
    #
    # 
    #    
    print('')

Fold #1


AttributeError: can't set attribute

In [ ]:
a = [0.7029411764705883,
0.6264705882352941,
#0.29090909090909095,
#0.5090909090909091,
#0.5378787878787878,
0.6636363636363636,
0.8272727272727274,
0.7393939393939394,
0.696969696969697,
0.7575757575757576]

np.mean(a)